## Dropout(드롭아웃)

드롭아웃은 특정 뉴런의 확률 p를 0으로 바꿔줌으로써 오버피팅을 최소화하는 방법이다.  
확률 p가 0이 되면 이전 층에서 계산된 값이 다음 층으로 전달되지 않게 된다.  
모델의 수용력 관점에서 보자면 각 층의 수용력을 p만큼 낮춰 전체 모델의 수용력을 줄이는 것이다.  
다른 관점에서 보자면 드롭아웃은 수용력이 낮은 모델들의 앙상블이라고도 할 수 있다.  
학습할 때 생기는 수용력이 낮은 모델들이 테스트시에는 드롭을 하지 않아 합쳐지기 때문이다.

#### module

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


#### parameters

In [3]:
num_epoch = 10
batch_size = 256
learning_rate = 2e-4

#### data

[pytorch dataset download error solution](https://github.com/pytorch/vision/issues/1938)

In [4]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [5]:
# download
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [6]:
mnist_train.__getitem__(0)[0].size(), mnist_train.__len__()

(torch.Size([1, 28, 28]), 60000)

In [7]:
mnist_test.__getitem__(0)[0].size(), mnist_test.__len__()

(torch.Size([1, 28, 28]), 10000)

In [8]:
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

#### model

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.Dropout2d(0.3),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*7*7, 100),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(100, 10)
        )
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

In [10]:
model = CNN().to(device)

#### loss

In [11]:
loss_func = nn.CrossEntropyLoss()

#### optimizer

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#### train

In [13]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()
        
    if i%10 == 0:
        print(loss)

tensor(2.3000, device='cuda:0', grad_fn=<NllLossBackward>)


#### test

In [14]:
correct = 0
total = 0

model.eval() # 배치정규화, 드롭아웃은 테스트시 학습과 다르게 동작하므로 model을 evaluation 모드로 바꿔야함
with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y = label.to(device)
        
        output = model.forward(x)
        _, output_index = torch.max(output, 1)
        
        total += label.size(0)
        correct += (output_index == y).sum().float()
        
    print(f"Accuracy of Test Data: {correct/total*100}")

Accuracy of Test Data: 10.096153259277344
